# Линейная полиномиальная регрессия

Предсказание расхода топлива автомобилем

<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Загрузка данных](#Загрузка-данных)
- [Предсказание расхода топлива](#Предсказание-расхода-топлива)
- [Источники](#Источники)

Увеличение области вывода:

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

Подключение библиотек:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

## Загрузка данных

In [ ]:
FILE_PATH = "../data/auto-mpg.data"

Признаки:
1. `mpg`: миль на галлон, действительное значение
2. `cylinders`: количество цилиндров, дискретное значение
3. `displacement`: объем двигателя, куб. дюймы, действительное значение
4. `horsepower`: horsepower: действительное значение
5. `weight`: вес автомобиля: lbs., действительное значение
6. `acceleration`: время разгона до 60 mph, сек., действительное значение
7. `model_year`: год выпуска модели, (по модулю 100), дискретное значение
8. `origin`: регион (1. American, 2. European, 3. Japanese), дискретное значение
9. `name`: наименование модели, строка (уникально для каждого экземпляра)

In [ ]:
CLMNS = [
    "mpg", "cylinders", "displacement", "horsepower", 
    "weight", "acceleration", "model_year", "origin", "name"
]

# Загрузка датасета
# Замечание: 
#  1) Разбиваем на столбцы по пробелам (один и более)
#  2) Там, где ?, заменяем на NaN
#  3) Удаляем строки с NaN

df = pd.read_csv(FILE_PATH, 
                 sep="\s+", 
                 names=CLMNS, 
                 na_values=["?",]).dropna()

df.head()

In [ ]:
# График
plt.figure(1, figsize=[12, 4])
plt.subplot(1,2,1)
plt.title("horsepower")
plt.scatter(df["horsepower"], df["mpg"], color="green")
plt.xlabel("$horsepower$")
plt.ylabel("$mpg$")
plt.grid(True)

plt.subplot(1,2,2)
plt.title("weight")
plt.scatter(df["weight"], df["mpg"], color="green")
plt.xlabel("$weight$")
plt.ylabel("$mpg$")
plt.grid(True)
plt.show()

## Предсказание расхода топлива

Модель 1:

$$h_1(x) = \theta_0 + \theta_1\cdot\text{horsepower}$$

Модель 2:

$$h_2(x) = \theta_0 + \theta_1\cdot\text{horsepower} + \theta_2\cdot\text{horsepower}^2$$

Модель 3:

$$h_3(x) = \theta_0 + \theta_1\cdot\text{horsepower} + \theta_2\cdot\text{weight} $$

Модель 4:

$$h_4(x) = \theta_0 + \theta_1\cdot\text{horsepower} + \theta_2\cdot\text{horsepower}^2 + \theta_3\cdot\text{weight} $$

In [ ]:
def create_poly_as_dataframe(df_train, df_test, degree):
    """
    Создает датафреймы с полиномами для обучающей и тестовой частей.
    
    Замечание: В данном случае нет необходимости создавать полиномы отдельно
    для обучающего и тестового датафреймов. Можно было бы это сделать для всего
    исходного датафрейма. Однако интерфейс PolynomialFeatures подразумевает
    использование методов fit и transform. Поэтому разделение применяется
    для соблюдения общего подхода.
    """
    pf = PolynomialFeatures(degree=degree)
    train_poly = pf.fit_transform(df_train)
    test_poly = pf.transform(df_test)
    return pd.DataFrame(train_poly, index=df_train.index), pd.DataFrame(test_poly, index=df_test.index)


def create_poly_as_matrix(df_train, df_test, degree):
    """
    Создает матрицы с полиномами для обучающей и тестовой частей.
    """
    pf = PolynomialFeatures(degree=degree)
    train_poly = pf.fit_transform(df_train)
    test_poly = pf.transform(df_test)
    return np.asmatrix(train_poly), np.asmatrix(test_poly)

In [ ]:
# # Формирование датафрейма признаков: 
# # исходный датафрейм признаков + датафрейм полиномов (кроме самих признаков и 1)
# poly_degree = 3
# num_poly_features = df_train_X[feature_clmns_1].columns.size

# df_train_poly, df_test_poly = create_poly_as_dataframe(df_train_X[feature_clmns_1], 
#                                                        df_test_X[feature_clmns_1], 
#                                                        degree=poly_degree)

# # Замечание: Используется цикл вместо concat, чтобы избежать
# # повторного включения столбцов
# for i in range(num_poly_features+1, df_train_poly.columns.size):
#     df_train_X[i] = df_train_poly[i]
    
# df_train_X.head()

In [ ]:
def plot_true_predicted(df_X, df_y, label_clmn="label", prediction_clmn="prediction", title=None):
    """
    Построение графиков действительных значений и предсказанных 
    по каждому признаку.
    """
    
    feature_names = df_X.columns
    num_features = feature_names.size
    num_plot_rows = np.ceil((num_features+1)/2.0)
    columns = df_X.columns
    
    fig = plt.figure(figsize=[12, 4*num_plot_rows])
    for i in range(num_features):
        plt.subplot(num_plot_rows, 2, i+1)
        plt.vlines(df_X[feature_names[i]], ymin=df_y[label_clmn], ymax=df_y[prediction_clmn], 
                   colors="black", linestyles="dotted", lw=1, zorder=1)
        plt.scatter(df_X[feature_names[i]], df_y[label_clmn], 
                    color="green", label="true", zorder=2)
        plt.scatter(df_X[feature_names[i]], df_y[prediction_clmn], 
                    color="red", label="predicted", zorder=3)
        plt.xlabel("$%s$" % feature_names[i])
        plt.ylabel("$%s$" % label_clmn)
        plt.legend()
        plt.grid(True) 
    
    plt.subplot(num_plot_rows, 2, num_features+1)
    plt.scatter(df_y[prediction_clmn], df_y[label_clmn], color="slategrey")
    xlim = plt.gca().get_xlim() 
    plt.plot(xlim, xlim, '--', color="grey")
    plt.xlim(xlim) 
    plt.xlabel("$\\bar{y}$")
    plt.ylabel("$y$")
    plt.grid(True) 
    
    plt.tight_layout()
    
#     if title:
#         plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
#         plt.suptitle(title,  y=.98, fontsize=16)
    plt.show()

In [ ]:
# Столбец целевого значения (действительного значения)
target_clmn = ["mpg"]

# Столбцы признаков (все кроме целевого значения)
all_feature_clmns = df.columns.delete(df.columns.get_loc(target_clmn[0]))

# Столбцы признаков для моделей
feature_clmns_1 = ["horsepower"]
feature_clmns_2 = ["horsepower", "horsepower^2"]
feature_clmns_3 = ["horsepower", "weight"]
feature_clmns_4 = ["horsepower", "horsepower^2", "weight"]

# Разбиение исходных данных на обучающее и тестовое множества
df_train_X, df_test_X, df_train_y, df_test_y = train_test_split(df[all_feature_clmns], df[target_clmn], 
                                                                test_size=0.3, random_state=1234)

# Добавление полинома в датафрейм признаков (в данном случае 
# нужна только степень 2 для horsepower)

# Вариант 1

# Добавление столбца (можно сделать и для всего датафрейма)
df_train_X["horsepower^2"] = df_train_X["horsepower"]**2
df_test_X["horsepower^2"] = df_test_X["horsepower"]**2


# Вариант 2 (с использованием PolynomialFeatures)

# poly_degree = 2

# train_poly_matrix, test_poly_matrix = create_poly_as_matrix(df_train_X[feature_clmns_1], 
#                                                             df_test_X[feature_clmns_1], 
#                                                             degree=poly_degree)
# df_train_X["horsepower^2"] = train_poly_matrix[:,2]
# df_test_X["horsepower^2"] = test_poly_matrix[:,2]


# Список столбцов признаков для всех моделей
features_set = [feature_clmns_1, feature_clmns_2, feature_clmns_3, feature_clmns_4]

# Обучение и оценка качества моделей
for indx, features in enumerate(features_set):
    
    # Обучение
    model = LinearRegression()
    model.fit(df_train_X[features], df_train_y[target_clmn[0]])
    
    # Параметры обученных моделей
    print("Model", indx + 1)
    print("\tw0 =", model.intercept_)
    for i, coef in enumerate(model.coef_):
        print("\tw{} = {}".format(i+1, coef))
    
    # Предсказания
    model_name = "model_{}_pred".format(indx+1)
    df_train_y[model_name] = model.predict(df_train_X[features])
    df_test_y[model_name] = model.predict(df_test_X[features])
    
    # Среднеквадратические ошибки на тестовом подмножестве для всех моделей
    mse = mean_squared_error(df_test_y[target_clmn], model.predict(df_test_X[features]))
    print("\tMSE = {}".format(mse))
    
    # Графики
    plot_true_predicted(df_test_X[features], 
                        df_test_y, 
                        label_clmn=target_clmn[0], 
                        prediction_clmn=model_name)

# Действительные и предсказанные значения для тестовых данных (первые пять)
df_test_y.head(5)

## Альтернативная реализация

In [ ]:
def transform_to_poly(df, clmn, inplace=True):
    """Трансформация"""
    clmn_new = "{}^2".format(clmn)
    if inplace:
        df[clmn_new] = df[clmn]**2
        return df
    return df.assign(**{clmn_new: df[clmn]**2})


def train_and_predit(df, model, feature_clmns, label_clmn, predicted_clmn="predicted", inplace=True):
    """Обучение и предсказание"""
    model.fit(df[feature_clmns], df[label_clmn])
    if inplace:
        df[predicted_clmn] = model.predict(df[feature_clmns])
        return df
    return df.assign(**{predicted_clmn: model.predict(df[feature_clmns])})


def predict(df, model, feature_clmns, predicted_clmn="predicted", inplace=True):
    """Предсказание"""
    if inplace:
        df[predicted_clmn] = model.predict(df[feature_clmns])
        return df
    return df.assign(**{predicted_clmn: model.predict(df[feature_clmns])})


def describe(df, name, model, feature_clmns, label_clmn, predicted_clmn="predicted"):
    """Вывод информации о модели"""
   
    # Наименование
    print(name)
    
    # Параметры обученных моделей
    print("\tw0 =", model.intercept_)
    for i, coef in enumerate(model.coef_):
        print("\tw{} = {}".format(i+1, coef))
        
    # Среднеквадратические ошибки на тестовом подмножестве для всех моделей
    mse = mean_squared_error(df[label_clmn], df[predicted_clmn])
    print("\tMSE = {}".format(mse))
    
    # Графики
    plot_true_predicted(df[feature_clmns], df, 
                       label_clmn=label_clmn, 
                       prediction_clmn=predicted_clmn)
    
    return df
    
    
# Разбиение исходных данных на обучающее и тестовое множества
df_train, df_test = train_test_split(df, test_size=0.3, random_state=1234)

# Столбцы признаков и целевого значения
target_clmn = "mpg"
feature_clmns = ["horsepower", "horsepower^2"]

# Инициализация модели
model = LinearRegression()   

# Обучение и тестирование
df_train__predicted = df_train\
    .pipe(transform_to_poly, "horsepower", False)\
    .pipe(train_and_predit, model, feature_clmns, target_clmn)\
    .pipe(describe, "Train", model, feature_clmns, target_clmn)

df_test__predicted = df_test\
    .pipe(transform_to_poly, "horsepower", False)\
    .pipe(predict, model, feature_clmns)\
    .pipe(describe, "Test", model, feature_clmns, target_clmn)

df_test__predicted[feature_clmns + [target_clmn] + ["predicted"]].head(5)

## Источники

[Auto MPG Data Set](https://archive.ics.uci.edu/ml/datasets/auto+mpg)